# Nguyễn Quốc Đạt 20231570

##  Câu 1: Sử dụng thư viện nltk để tách từ trong câu hoặc đoạn

In [1]:
import nltk

# đổi nltk.data.path trỏ theo đúng setup trên windows, đổi qua linux thì bỏ
nltk.data.path.append('F:/nltk_data')

from nltk.data import load
tokenizer = load('tokenizers/punkt/english.pickle')
print(tokenizer.tokenize("Hello world. How are you?"))

['Hello world.', 'How are you?']


In [6]:
from nltk.tokenize import word_tokenize

sentence = "Hello world. How are you? What have you been doing?"
words = word_tokenize(sentence)
print(words)

['Hello', 'world', '.', 'How', 'are', 'you', '?', 'What', 'have', 'you', 'been', 'doing', '?']


In [7]:
from nltk.probability import FreqDist

# tính toán tần suất xuất hiện của các từ
freq_dist = FreqDist(words)
print(freq_dist)

# in ra
for word, frequency in freq_dist.items():
    print(f'{word}: {frequency}')

<FreqDist with 11 samples and 13 outcomes>
Hello: 1
world: 1
.: 1
How: 1
are: 1
you: 2
?: 2
What: 1
have: 1
been: 1
doing: 1


In [8]:
# xác định từ muốn dừng
stop_words = {'.', '?'}

# loại bỏ và in ra những từ ko phải stop
filtered_words = [word for word in words if word not in stop_words]
print(filtered_words)

['Hello', 'world', 'How', 'are', 'you', 'What', 'have', 'you', 'been', 'doing']


In [ ]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

# tìm từ gốc của từng từ trong danh sách filtered_words
lemmas = [lemmatizer.lemmatize(word) for word in filtered_words]
print(lemmas)

# có lẽ ví dụ đưa ra phía trên chưa rõ ràng

['Hello', 'world', 'How', 'are', 'you', 'What', 'have', 'you', 'been', 'doing']


## Câu 2: Viết hàm/chương trình chuẩn hoá 1 đoạn văn Tiếng Việt
### Ví dụ đầu vào, đầu ra của hàm/chương trình như sau:
Đoạn văn đầu vào: "Tỷ giá trung tâm giữa đồng Việt Nam (VND) và đô la Mỹ (USD) được NHNN công bố hôm 22/9 ở mức 24.060 đồng mỗi USD. Giá USD tại các ngân hàng thương mại hôm 23/9 được giao dịch quanh mức 24.160-23.530 đồng/USD (mua vào bán ra)." <br>

Đoạn văn đầu ra: "tỷ giá trung tâm giữa đồng việt nam vê nờ đê và đô la mỹ u ét đê được ngân hàng nhà nước công bố hôm hai mươi hai tháng chín ở mức hai mươi không trăm sáu mươi đồng mỗi u ét đê. giá u ét đê tại các ngân hàng thương mại hôm hai mươi ba tháng chín được giao dịch mức hai mươi nghìn một trăm sáu mươi tới hai mươi tư nghìn năm trăm ba mưới đồng trên u ét đê mua vào và bán ra "

### Hướng giải quyết bài toán
Có thể thấy việc chuẩn hoá yêu cầu trong bài toán cũng đã có bôi đỏ và xanh những phần cần chú ý trong file PDF khi đồng hoá toàn bộ cần viết thường, giải nghĩa các từ viết tắt và số tiền được đề cập, đồng thời có 1 số ký hiệu như '-' hay '/' cũng cần đổi về dạng từ

In [1]:
import re

# bảng thay thế viết tắt
VIETNAMESE_ABBREVIATIONS = {
    "NHNN": "ngân hàng nhà nước",
    "VND": "việt nam vê nờ đê",
    "USD": "u ét đê",
}

# chuyển số nguyên sang chữ tiếng Việt
def number_to_vietnamese(n):
    units = ["", "một", "hai", "ba", "bốn", "năm", "sáu", "bảy", "tám", "chín"]
    tens = ["", "mười", "hai mươi", "ba mươi", "bốn mươi", "năm mươi", "sáu mươi", "bảy mươi", "tám mươi", "chín mươi"]
    
    if n < 10:
        return units[n]
    elif n < 100:
        return tens[n // 10] + (" " + units[n % 10] if n % 10 != 0 else "")
    elif n < 1000:
        return units[n // 100] + " trăm " + (number_to_vietnamese(n % 100) if n % 100 != 0 else "")
    elif n < 1000000:
        return number_to_vietnamese(n // 1000) + " nghìn " + (number_to_vietnamese(n % 1000) if n % 1000 != 0 else "")

# hàm chuẩn hóa văn bản tiếng Việt
def normalize_vietnamese_text(text):
    # chuyển về chữ thường
    text = text.lower()

    # thay thế các viết tắt
    for abbr, full in VIETNAMESE_ABBREVIATIONS.items():
        text = text.replace(abbr.lower(), full)

    # chuyển đổi ngày tháng dạng "22/9" thành "hai mươi hai tháng chín"
    text = re.sub(r'(\d{1,2})/(\d{1,2})', lambda m: f"{number_to_vietnamese(int(m.group(1)))} tháng {number_to_vietnamese(int(m.group(2)))}", text)

    # chuyển đổi số thập phân (dấu `.`) sang chữ
    text = re.sub(r'(\d+)\.(\d+)', lambda m: f"{number_to_vietnamese(int(m.group(1)))} không trăm {number_to_vietnamese(int(m.group(2)))}", text)

    # chuyển đổi số nguyên sang chữ
    text = re.sub(r'\b\d+\b', lambda m: number_to_vietnamese(int(m.group(0))), text)

    # chuẩn hóa dấu câu, khoảng trắng
    text = re.sub(r'[^a-zA-Z0-9 àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]', ' ', text)  # Giữ lại chữ và số
    """
    lý do để có 1 dòng regex dài như vậy nhằm giữ lại các ký tự tiếng Việt từ các nguyên âm chính 'aeiouy' và các biến thể + trường hợp đặc biệt 'đ', số và khoảng trắng, còn các ký tự khác sẽ bị bỏ qua thay thế bằng khoảng trắng
    chi tiết cụ thể được hướng dẫn qua 1 bài viết: https://www.facebook.com/photo.php?fbid=357731432600303&id=113963433643772&set=a.115372476836201&locale=vi_VN
    """ 
    text = re.sub(r'\s+', ' ', text).strip()  # Xóa khoảng trắng thừa
    
    return text

# ví dụ sử dụng
input_text = "Tỷ giá trung tâm giữa đồng Việt Nam (VND) và đô la Mỹ (USD) được NHNN công bố hôm 22/9 ở mức 24.060 đồng mỗi USD. Giá USD tại các ngân hàng thương mại hôm 23/9 được giao dịch quanh mức 24.160-23.530 đồng/USD (mua vào bán ra)."
output_text = normalize_vietnamese_text(input_text)

print(output_text)


tỷ giá trung tâm giữa đồng việt nam việt nam vê nờ đê và đô la mỹ u ét đê được ngân hàng nhà nước công bố hôm hai mươi hai tháng chín ở mức hai mươi bốn không trăm sáu mươi đồng mỗi u ét đê giá u ét đê tại các ngân hàng thương mại hôm hai mươi ba tháng chín được giao dịch quanh mức hai mươi bốn không trăm một trăm sáu mươi hai mươi ba không trăm năm trăm ba mươi đồng u ét đê mua vào bán ra
